In [2]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores

##Best config is 180: {'topK': 494, 'alpha': 0.1597863887003789}

In [3]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [4]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13933 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13897 ( 0.6%) Users that have less than 1 test interactions


In [5]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [6]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=stacked_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 1 ( 0.0%) users with no interactions.


In [7]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [10:55<00:00, 55.08it/s]

In [8]:
similarity_rp3 = RP3Beta_recommender.W_sparse
similarity_slime = SLIME_recommender.W_sparse

In [9]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real,Integer

hyperparameters_range_dictionary = {
    "topK": Integer(10,500),
    "alpha": Real(0.0, 1.0)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_rp3, similarity_slime],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 340, 'alpha': 0.31432839939201845}
ItemKNNSimilarityHybridRecommender: URM Detected 1 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13897 (100.0%) in 11.27 sec. Users per second: 1233
SearchBayesianSkopt: New best config found. Config 0: {'topK': 340, 'alpha': 0.31432839939201845} - results: PRECISION: 0.2398431, PRECISION_RECALL_MIN_DEN: 0.2452274, RECALL: 0.0707194, MAP: 0.1190915, MAP_MIN_DEN: 0.1215448, MRR: 0.4694508, NDCG: 0.2502804, F1: 0.1092312, HIT_RATE: 0.9083975, ARHR_ALL_HITS: 0.7439242, NOVELTY: 0.0057649, AVERAGE_POPULARITY: 0.4630976, DIVERSITY_MEAN_INTER_LIST: 0.9341249, DIVERSITY_HERFINDAHL: 0.9934058, COVERAGE_ITEM: 0.1911512, COVERAGE_ITEM_CORRECT: 0.1050446, COVERAGE_USER: 0.9937786, COVERAGE_USER_CORRECT: 0.9027460, DIVERSITY_GINI: 0.0225884, SHANNON_ENTROPY: 8.6742817, RATIO_DIVERSITY_HERFINDAHL: 0.9937721, RATIO_DIVERSITY_GINI: 0.0845558,

100%|████████████████████████████████████▉| 18056/18059 [11:10<00:00, 55.08it/s]

EvaluatorHoldout: Processed 13933 (100.0%) in 10.44 sec. Users per second: 1335
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'topK': 340, 'alpha': 0.31432839939201845} - results:
CUTOFF: 10 - PRECISION: 0.2798105, PRECISION_RECALL_MIN_DEN: 0.2844636, RECALL: 0.0709255, MAP: 0.1463642, MAP_MIN_DEN: 0.1485315, MRR: 0.5126228, NDCG: 0.2898681, F1: 0.1131660, HIT_RATE: 0.9363382, ARHR_ALL_HITS: 0.8608289, NOVELTY: 0.0057663, AVERAGE_POPULARITY: 0.4630592, DIVERSITY_MEAN_INTER_LIST: 0.9340090, DIVERSITY_HERFINDAHL: 0.9933942, COVERAGE_ITEM: 0.1947505, COVERAGE_ITEM_CORRECT: 0.1152888, COVERAGE_USER: 0.9963530, COVERAGE_USER_CORRECT: 0.9329233, DIVERSITY_GINI: 0.0228013, SHANNON_ENTROPY: 8.6790854, RATIO_DIVERSITY_HERFINDAHL: 0.9937605, RATIO_DIVERSITY_GINI: 0.0853529, RATIO_SHANNON_ENTROPY: 0.6954287, RATIO_AVERAGE_POPULARITY: 2.3833131, RATIO_NOVELTY: 0.0304515, 


SearchBayesianSkopt: Saving model in logs/ItemKNNSimilarityHybridRecommender

ItemKNNSimilarityHybridRe

100%|████████████████████████████████████▉| 18056/18059 [11:40<00:00, 25.78it/s]

EvaluatorHoldout: Processed 13897 (100.0%) in 10.32 sec. Users per second: 1347


...

Iteration No: 173 ended. Search finished for the next optimal point.
Time taken: 14.1313
Function value obtained: -0.0994
Current minimum: -0.1200
Iteration No: 174 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 93, 'alpha': 0.589221743633949}
ItemKNNSimilarityHybridRecommender: URM Detected 1 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13897 (100.0%) in 8.04 sec. Users per second: 1728
SearchBayesianSkopt: Config 173 is suboptimal. Config: {'topK': 93, 'alpha': 0.589221743633949} - results: PRECISION: 0.2330791, PRECISION_RECALL_MIN_DEN: 0.2392538, RECALL: 0.0701848, MAP: 0.1144943, MAP_MIN_DEN: 0.1173632, MRR: 0.4618349, NDCG: 0.2439513, F1: 0.1078837, HIT_RATE: 0.9061668, ARHR_ALL_HITS: 0.7240151, NOVELTY: 0.0058577, AVERAGE_POPULARITY: 0.4436143, DIVERSITY_MEAN_INTER_LIST: 0.9283951, DIVERSITY_HERFINDAHL: 0.9928328, COVERAGE_ITEM: 0.2073758, COVERAGE_ITEM_CORRECT: 0.1198848, COVERAGE_USER: 0.9937786, COVERAGE_USER_CO

Iteration No: 179 ended. Search finished for the next optimal point.
Time taken: 14.1967
Function value obtained: -0.1200
Current minimum: -0.1200
Iteration No: 180 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 490, 'alpha': 0.170747162687608}
ItemKNNSimilarityHybridRecommender: URM Detected 1 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13897 (100.0%) in 10.59 sec. Users per second: 1312
SearchBayesianSkopt: Config 179 is suboptimal. Config: {'topK': 490, 'alpha': 0.170747162687608} - results: PRECISION: 0.2401094, PRECISION_RECALL_MIN_DEN: 0.2450960, RECALL: 0.0701755, MAP: 0.1199013, MAP_MIN_DEN: 0.1222663, MRR: 0.4719760, NDCG: 0.2509849, F1: 0.1086086, HIT_RATE: 0.9086134, ARHR_ALL_HITS: 0.7479728, NOVELTY: 0.0057606, AVERAGE_POPULARITY: 0.4576216, DIVERSITY_MEAN_INTER_LIST: 0.9431929, DIVERSITY_HERFINDAHL: 0.9943125, COVERAGE_ITEM: 0.1879949, COVERAGE_ITEM_CORRECT: 0.1032172, COVERAGE_USER: 0.9937786, COVERAGE_USER

Iteration No: 185 ended. Search finished for the next optimal point.
Time taken: 14.4948
Function value obtained: -0.1168
Current minimum: -0.1200
Iteration No: 186 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 497, 'alpha': 0.15823426150175862}
ItemKNNSimilarityHybridRecommender: URM Detected 1 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13897 (100.0%) in 10.54 sec. Users per second: 1319
SearchBayesianSkopt: Config 185 is suboptimal. Config: {'topK': 497, 'alpha': 0.15823426150175862} - results: PRECISION: 0.2402821, PRECISION_RECALL_MIN_DEN: 0.2452708, RECALL: 0.0702011, MAP: 0.1200274, MAP_MIN_DEN: 0.1223921, MRR: 0.4722422, NDCG: 0.2511457, F1: 0.1086569, HIT_RATE: 0.9083975, ARHR_ALL_HITS: 0.7484624, NOVELTY: 0.0057624, AVERAGE_POPULARITY: 0.4564260, DIVERSITY_MEAN_INTER_LIST: 0.9439958, DIVERSITY_HERFINDAHL: 0.9943928, COVERAGE_ITEM: 0.1879949, COVERAGE_ITEM_CORRECT: 0.1032726, COVERAGE_USER: 0.9937786, COVERAGE_

Iteration No: 192 ended. Search finished for the next optimal point.
Time taken: 14.1580
Function value obtained: -0.1193
Current minimum: -0.1200
Iteration No: 193 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 380, 'alpha': 0.1793192284414837}
ItemKNNSimilarityHybridRecommender: URM Detected 1 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13897 (100.0%) in 10.56 sec. Users per second: 1316
SearchBayesianSkopt: Config 192 is suboptimal. Config: {'topK': 380, 'alpha': 0.1793192284414837} - results: PRECISION: 0.2399870, PRECISION_RECALL_MIN_DEN: 0.2450032, RECALL: 0.0701659, MAP: 0.1196943, MAP_MIN_DEN: 0.1220638, MRR: 0.4709427, NDCG: 0.2506992, F1: 0.1085845, HIT_RATE: 0.9079657, ARHR_ALL_HITS: 0.7465783, NOVELTY: 0.0057708, AVERAGE_POPULARITY: 0.4537967, DIVERSITY_MEAN_INTER_LIST: 0.9437642, DIVERSITY_HERFINDAHL: 0.9943696, COVERAGE_ITEM: 0.1890470, COVERAGE_ITEM_CORRECT: 0.1042140, COVERAGE_USER: 0.9937786, COVERAGE_US

Iteration No: 199 ended. Search finished for the next optimal point.
Time taken: 14.7949
Function value obtained: -0.1200
Current minimum: -0.1200
Iteration No: 200 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 494, 'alpha': 0.1644082453421909}
ItemKNNSimilarityHybridRecommender: URM Detected 1 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13897 (100.0%) in 10.76 sec. Users per second: 1291
SearchBayesianSkopt: Config 199 is suboptimal. Config: {'topK': 494, 'alpha': 0.1644082453421909} - results: PRECISION: 0.2401382, PRECISION_RECALL_MIN_DEN: 0.2451269, RECALL: 0.0701858, MAP: 0.1199739, MAP_MIN_DEN: 0.1223407, MRR: 0.4723093, NDCG: 0.2510677, F1: 0.1086238, HIT_RATE: 0.9083975, ARHR_ALL_HITS: 0.7483828, NOVELTY: 0.0057615, AVERAGE_POPULARITY: 0.4570471, DIVERSITY_MEAN_INTER_LIST: 0.9435953, DIVERSITY_HERFINDAHL: 0.9943527, COVERAGE_ITEM: 0.1881610, COVERAGE_ITEM_CORRECT: 0.1032726, COVERAGE_USER: 0.9937786, COVERAGE_US